## Streaming linear regression

When data arrive in a streaming fashion, it is useful to fit regression models online, updating the parameters of the model as new data arrives. spark.mllib currently supports streaming linear regression using ordinary least squares. The fitting is similar to that performed offline, except fitting occurs on each batch of data, so that the model continually updates to reflect the data from the stream.

First, we import the necessary classes for parsing our input data and creating the model.

In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD

Then we make input streams for training and testing data. We assume a StreamingContext ssc has already been created, see Spark Streaming Programming Guide for more info. For this example, we use labeled points in training and testing streams, but in practice you will likely want to use unlabeled vectors for test data.

In [2]:
def parse(lp):
    label = float(lp[lp.find('(') + 1: lp.find(',')])
    vec = Vectors.dense(lp[lp.find('[') + 1: lp.find(']')].split(','))
    return LabeledPoint(label, vec)

In [4]:
trainingData = ssc.textFileStream("/tmp/training/data/dir").map(parse).cache()
testData = ssc.textFileStream("/tmp/testing/data/dir").map(parse)



NameError: name 'ssc' is not defined

We create our model by initializing the weights to 0

In [ ]:
numFeatures = 3
model = StreamingLinearRegressionWithSGD()
model.setInitialWeights([0.0, 0.0, 0.0])

Now we register the streams for training and testing and start the job.

In [5]:
model.trainOn(trainingData)
print(model.predictOnValues(testData.map(lambda lp: (lp.label, lp.features))))

ssc.start()
ssc.awaitTermination()

NameError: name 'model' is not defined

We can now save text files with data to the training or testing folders. Each line should be a data point formatted as (y,[x1,x2,x3]) where y is the label and x1,x2,x3 are the features. Anytime a text file is placed in /training/data/dir the model will update. Anytime a text file is placed in /testing/data/dir you will see predictions. As you feed more data to the training directory, the predictions will get better!